In [18]:
import tensorflow as tf
import numpy as np
import pandas as pd
import json
import datetime
from datetime import datetime, timedelta

In [2]:
# Load data makanan
with open("food_data.json", "r") as f:
    food_data = json.load(f)

# Konversi ke DataFrame
food_df = pd.DataFrame(food_data)

# Load tracker memory
with open("result/tracker_memory.json", "r") as f:
    tracker_memory = json.load(f)

In [3]:
# Filter makanan yang sering dimakan dalam 1 bulan terakhir
def get_frequent_foods(tracker_memory, days=30):
    one_month_ago = datetime.datetime.now() - datetime.timedelta(days=days)
    frequent_foods = {}

    for date, entries in tracker_memory.items():
        date_obj = datetime.datetime.strptime(date, "%Y-%m-%d")
        if date_obj >= one_month_ago:
            for entry in entries:
                food_name = entry["food_name"]
                if food_name in frequent_foods:
                    frequent_foods[food_name] += 1
                else:
                    frequent_foods[food_name] = 1

    # Sort by frequency
    sorted_foods = sorted(frequent_foods.items(), key=lambda x: x[1], reverse=True)
    return [food[0] for food in sorted_foods]

frequent_foods = get_frequent_foods(tracker_memory)

In [12]:
# TensorFlow Model for Recommendation
def build_recommendation_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(32, activation="relu", input_shape=(5,)),  # protein, fat, carbs, calories, frequency
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(1, activation="linear")  # Score output
    ])
    model.compile(optimizer="adam", loss="mse", metrics=["mae"])
    return model

# Dataset Preparation for Model
def prepare_dataset(food_df, target_protein, target_calories, frequent_foods):
    data = []
    labels = []

    for index, row in food_df.iterrows():
        frequency = frequent_foods.count(row["name"]) if row["name"] in frequent_foods else 0
        input_features = [row["proteins"], row["fat"], row["carbohydrate"], row["calories"], frequency]
        score = (row["proteins"] / target_protein) + (row["calories"] / target_calories)  # Example scoring
        data.append(input_features)
        labels.append(score)

    return np.array(data), np.array(labels)


In [13]:
# Example User Target Nutrients
user_target_protein = 60  # gram
user_target_calories = 2000  # kcal

In [14]:
# Prepare Data
data, labels = prepare_dataset(food_df, user_target_protein, user_target_calories, frequent_foods)

# Train Model
model = build_recommendation_model()
model.fit(data, labels, epochs=50, verbose=1)

# Save Model
model.save("result/recommendation_model.h5")

Epoch 1/50
43/43 [==============================] - 1s 2ms/step - loss: 1275.9729 - mae: 20.4680
Epoch 2/50
43/43 [==============================] - 0s 2ms/step - loss: 37.2939 - mae: 3.8331
Epoch 3/50
43/43 [==============================] - 0s 3ms/step - loss: 16.9361 - mae: 2.5334
Epoch 4/50
43/43 [==============================] - 0s 3ms/step - loss: 6.8976 - mae: 1.5190
Epoch 5/50
43/43 [==============================] - 0s 4ms/step - loss: 3.0970 - mae: 1.0779
Epoch 6/50
43/43 [==============================] - 0s 3ms/step - loss: 1.8324 - mae: 0.8025
Epoch 7/50
43/43 [==============================] - 0s 3ms/step - loss: 1.2632 - mae: 0.6685
Epoch 8/50
43/43 [==============================] - 0s 3ms/step - loss: 0.9033 - mae: 0.5743
Epoch 9/50
43/43 [==============================] - 0s 3ms/step - loss: 0.6872 - mae: 0.5063
Epoch 10/50
43/43 [==============================] - 0s 5ms/step - loss: 0.6646 - mae: 0.4950
Epoch 11/50
43/43 [==============================] - 0s 3ms/ste

In [17]:
# Generate Recommendation
def generate_recommendation(model_path, target_protein, target_calories, food_df, frequent_foods):
    model = tf.keras.models.load_model(model_path)
    data, _ = prepare_dataset(food_df, target_protein, target_calories, frequent_foods)
    predictions = model.predict(data)

    # Combine with food data
    food_df["score"] = predictions.flatten()
    food_df = food_df.sort_values("score", ascending=False)

    # Add serving size
    food_df["serving"] = food_df["proteins"].apply(lambda x: int((target_protein / x) * 100) if x > 0 else 100)

    return food_df[["name", "score", "serving"]].head(10)

recommendations = generate_recommendation(
    "result/recommendation_model.h5", user_target_protein, user_target_calories, food_df, frequent_foods
)

print(recommendations)

43/43 [==============================] - 0s 3ms/step
                            name     score  serving
1208                         Teh  2.384814      307
1027                        Ragi  1.870380      139
460             Ikan kayu kering  1.844852       84
1329              Umbi Uwi segar  1.739010     2142
801              Kue Pacar Cina   1.520333     8571
1324         Udang kering mentah  1.507408       95
742         Kerupuk Kulit kerbau  1.442134       72
477   Ikan Mujair dendeng mentah  1.441603       87
320       Dendeng mujahir goreng  1.439139       80
476   Ikan Mujair dendeng goreng  1.439139       80
